In [18]:
# Querying GitHub API for open Dependabot Alerts #####
import os
import requests
import pandas as pd
import pprint as pp
import tabulate as tb

# Functions
def get_header(token):
    auth="Bearer " + token
    authdict={"Authorization": auth}
    return authdict

def get_api_url():
    if "GITHUB_API_URL" in os.environ:
        url = os.environ["GITHUB_API_URL"]
    else:
        url = 'https://api.github.com/graphql'
    
    return url

# A simple function to use requests.post to make the API call. Note the json= section.
def run_query(query,token): 
    head=get_header(token)
    apiURL=get_api_url()
    request = requests.post(apiURL, json={'query': query}, headers=head)
    if request.status_code == 200:
        response=request.json()
        return response
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))

def get_alerts(repo,owner,token): #  A simple function to use requests.post to make the API call. Note the json= section.
    # TODO - get around the pagination limits for accurate total issues
    # The GraphQL query (with a few aditional bits included) itself defined as a multi-line string.       
    query = """
    {
        repository(name: "REPO_NAME", owner: "REPO_OWNER") {
            vulnerabilityAlerts(first: 100,states: OPEN) {
                nodes {
                    state
                    createdAt
                    dismissedAt
                    state
                    securityVulnerability {
                        severity
                    }
                }
            }
        }
    }
    """
    # Parameterize the name/owner of the repo (TODO - multi-line f string)
    query=query.replace("REPO_NAME",repo)
    query=query.replace("REPO_OWNER",owner)
    # Query GitHub API
    result=run_query(query,token)
    pp.pprint(result)
    # Flatten into a dataframe
    rows=result['data']['repository']['vulnerabilityAlerts']['nodes']
    rows=pd.json_normalize(rows)
    # Append into an empty state to handle 0 rows
    alerts=pd.DataFrame(columns = ['state','securityVulnerability.severity','createdAt','dismissedAt'])
    alerts = alerts.append(rows)
    alerts=alerts.rename(columns={"securityVulnerability.severity": "severity"})
    # Return the number of alerts to console
    return alerts

def main():
    # Get inputs from envars (GitHub converts all inputs into INPUT_<UPPER CASE OF INPUT>)
    token = os.environ["INPUT_GITHUB_PERSONAL_TOKEN"]
    owner = os.environ["GITHUB_REPOSITORY_OWNER"]
    repo = os.environ["GITHUB_REPOSITORY"]
    repoName = repo.split("/")[-1]                      #  Cleans the in-case we get 'owner/repo' format
    # Query GitHub for full alerts breakdown
    alerts=get_alerts(repoName,owner,token)
    pp.pprint(alerts)
    # Breakdown stats
    statsDict={"total_alerts": len(alerts)}
    statsDict['critical_alerts']=len(alerts.loc[alerts['severity'] == 'CRITICAL'])
    statsDict['high_alerts']=len(alerts.loc[alerts['severity'] == 'HIGH'])
    statsDict['moderate_alerts']=len(alerts.loc[alerts['severity'] == 'MODERATE'])
    statsDict['low_alerts']=len(alerts.loc[alerts['severity'] == 'LOW'])
    pp.pprint(statsDict)
    #Set Outputs
    print(f"::set-output name=total_alerts::{statsDict['total_alerts']}")
    print(f"::set-output name=critical_alerts::{statsDict['critical_alerts']}")
    print(f"::set-output name=high_alerts::{statsDict['high_alerts']}")
    print(f"::set-output name=moderate_alerts::{statsDict['moderate_alerts']}")
    print(f"::set-output name=low_alerts::{statsDict['low_alerts']}")
    #Create markdown summary
    summaryFile = os.environ["GITHUB_STEP_SUMMARY"]  #https://docs.github.com/en/actions/using-workflows/workflow-commands-for-github-actions#adding-a-job-summary
    summary = {'Severity': ['CRITICAL','HIGH','MODERATE','LOW'], 'Open Issues': list( map(statsDict.get,['critical_alerts','high_alerts','moderate_alerts','low_alerts']))}
    summary = pd.DataFrame(data=summary)
    summary=summary.set_index('Severity')
    summaryMD=summary.to_markdown()
    summaryText=f"## ⚠ Open Dependabot Alerts\n There are currently {statsDict['total_alerts']} open security [vulnerabilities](https://github.com/{repo}/security/dependabot).\n"
    with open(summaryFile, "a") as myfile:
        myfile.write(summaryText)
        myfile.write(summaryMD)

In [19]:
os.environ["INPUT_GITHUB_PERSONAL_TOKEN"] = ""
os.environ["GITHUB_REPOSITORY_OWNER"] = "spicyparrot"
os.environ["GITHUB_REPOSITORY"] = "spicyparrot/check-dependabot"
get_api_url()

'https://api.github.com/graphql'

In [20]:
token = os.environ["INPUT_GITHUB_PERSONAL_TOKEN"]
owner = os.environ["GITHUB_REPOSITORY_OWNER"]
repo = os.environ["GITHUB_REPOSITORY"]
repoName = repo.split("/")[-1]
alerts=get_alerts(repoName,owner,token)

{'data': {'repository': {'vulnerabilityAlerts': {'nodes': []}}}}


/tmp/ipykernel_13/1142670249.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  alerts = alerts.append(rows)


In [6]:

# TODO - get around the pagination limits for accurate total issues
# The GraphQL query (with a few aditional bits included) itself defined as a multi-line string.       
query = """
{
    repository(name: "REPO_NAME", owner: "REPO_OWNER") {
        vulnerabilityAlerts(first: 100,states: OPEN) {
            nodes {
                state
                createdAt
                dismissedAt
                state
                securityVulnerability {
                    severity
                }
            }
        }
    }
}
"""
# Parameterise the name/owner of the repo (TODO - multi-line f string)
query=query.replace("REPO_NAME",repo)
query=query.replace("REPO_OWNER",owner)
pp.pprint(query)
result=run_query(query,token)


('\n'
 '{\n'
 '    repository(name: "spicyparrot/check-dependabot", owner: "spicyparrot") '
 '{\n'
 '        vulnerabilityAlerts(first: 100,states: OPEN) {\n'
 '            nodes {\n'
 '                state\n'
 '                createdAt\n'
 '                dismissedAt\n'
 '                state\n'
 '                securityVulnerability {\n'
 '                    severity\n'
 '                }\n'
 '            }\n'
 '        }\n'
 '    }\n'
 '}\n')


Exception: Query failed to run by returning code of 404. 
{
    repository(name: "spicyparrot/check-dependabot", owner: "spicyparrot") {
        vulnerabilityAlerts(first: 100,states: OPEN) {
            nodes {
                state
                createdAt
                dismissedAt
                state
                securityVulnerability {
                    severity
                }
            }
        }
    }
}


In [16]:

head=get_header(token)
apiURL=get_api_url()
request = requests.post(apiURL, json={'query': query}, headers=head)
response=request.json()
request.json()


#if request.status_code == 200:
#    
#    return response
#else:
#    raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))

{'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest'}